# 10  SOME SIMPLE ALGORITHMS AND DATA STRUCTURES

The goal of this chapter is to help you develop some general intuitions about

* <b>how to approach questions of efficiency</b>.

The major point was that the key to efficiency is

* a <b>good algorithm</b>,

* not <b>clever coding tricks</b>.

What we do instead is learn to <b>reduce</b> the most complex aspects of the problems with which we are faced <b>to previously solved problems</b>. 

More specifically, we:

* Develop an <b>understanding of the inherent complexity</b> of the problem with which we are faced,

* Think about how to **break** that problem up <b>into subproblems</b> 

* Relate those subproblems to other problems for which <b>efficient algorithms already exist</b>

**Keep in mind ：**

* The **most efficient** algorithm is **not always** the algorithm of **choice.**

* A program that does everything in the most efficient possible way is often

   * <b>needlessly difficult to understand</b>.

It is often **a good strategy** to :
   
* **start** by solving the problem at hand in the most **straightforward manner possible**

* instrument it to **find** any computational **bottlenecks**

* look for ways to <b>improve</b> the computational complexity of those parts of the program contributing to the bottlenecks.
 


## 10.1 Search Algorithms

A search algorithm is a method for finding an item or group of items with specific properties within a collection of items. 

We refer to the collection of items as a <b>search space</b>. 

The search space might be something concrete, such as a set of electronic medical records, or something abstract, such as the set of all integers.

In this section, we will examine two algorithms for searching a list. 

Each meets **the specification:**

```python
def search(L, e):
"""Assumes L is a list.
   Returns True if e is in L and False otherwise"""
```

The astute reader might wonder if this is not semantically equivalent to the Python expression
```python
e in L
``` 
The answer is yes, it is. 

If one is unconcerned about the efficiency of discovering whether **e is in L**, one should simply write that expression.

### 10.1.1 Linear Search and Using <font color='blue'>Indirection</font> to Access Elements

Python uses the following algorithm to determine if an element is in a list
```python
def search(L, e):
    for i in range(len(L)): # O(len(L))
        if L[i] == e:
            return True
    return False
```

If the element **e** is not in the list the algorithm will perform <b>O(len(L))</b> tests

* the complexity is <b>at best linear</b> in the length of L.

** Why “at best” linear? **

It will be linear **only if** : each operation **inside the loop can be **done** in **constant time**.

Let’s start by considering the simple case:

*  each element of the list is an **integer**

In this case the **address** in memory of the ith element of the list is simply

* $start + 4i$

where **start** is the address of the start of the list, integer variable occupy 4 bytes. 

Therefore we can assume that Python could compute the address of the ith element of a list of integers in constant time

In Python, a **list** is represented as a **length** (the number of objects in the list) and a sequence of **fixed-size pointers** to **objects**.

The Figure illustrates the use of these pointers.

* The shaded region represents a list containing four elements.

* <font color="red">The leftmost shaded box</font> contains a pointer to an integer indicating the length of the list.

* <font color="blue"><b>Each of the other shaded boxes</font> contains a pointer to an **object** in the list.

<img src="./img/10.1.1.PNG"/>

**IF** 

* the length field is four units of memory

* each pointer (address) occupies four units of memory

the address of the ith element of the list is stored at the address

$start + 4 + 4i$

this address can be found in constant time, and then the value stored at that address can be used to access the ith element. This access too is a constant-time operation.

This example illustrates one of the most important implementation techniques  used in computing: 

* <b>indirection</b>
  
   * Generally speaking, indirection involves accessing something by 
   
     **first accessing something else** that contains <b>a reference</b> to the thing initially sought.
     
This is what happens each time we use a variable to refer to the object to which that variable is bound. 

When we use a variable to **access a list** and then a reference stored in that list to **access another object**, we are going through two levels of indirection.


In [1]:
%%file ./code/ds/LinearSearch.c

/* Search an array for the given key using Linear Search (LinearSearch.c) */
#include <stdio.h>
#include <stdlib.h> 

int linearSearch(const int a[], int size, int key);

// Search the array for the given key
// If found, return array index [0, size-1]; otherwise, return size
int linearSearch(const int a[], int size, int key) {
   for (int i = 0; i < size; ++i) {
      if (a[i] == key) return i;
   }
   return size;
}
 

int main() {
   const int SIZE = 8;
   int a1[8] = {8, 4, 5, 3, 2, 9, 4, 1};
 
   int keys[3]={8,4,99};
   for(int i=0; i<3; i++) 
       printf("%d's index is: %d \n",keys[i],linearSearch(a1,SIZE, keys[i]));
   
   return 0; 
 }
 


Overwriting ./code/ds/LinearSearch.c


In [2]:
!gcc -o ./code/ds/LinearSearch.exe ./code/ds/LinearSearch.c

In [3]:
!.\code\ds\LinearSearch.exe 

8's index is: 0 
4's index is: 1 
99's index is: 8 


### 10.1.2 Binary Search and Exploiting Assumptions

Getting back to the problem of implementing 

**search(L, e)**, is **O(len(L))**。

The best we can do? Yes!

* If we know <b>nothing about the relationship of the values</b> of the elements in the list and the order in which they are stored.

But suppose we **know** something about the **order** in which elements are stored, 

  * we have a list of integers stored in <b>ascending order</b>.

We could change the implementation so that the search stops when it reaches a number larger than the number for which it is searching:

```python
def search(L, e):
    """Assumes L is a list, the elements of which are in
       ascending order.
       Returns True if e is in L and False otherwise"""
    for i in range(len(L)):
        if L[i] == e:
            return True
        if L[i] > e:  # ascending order
            return False
    return False
```

This would improve the average running time. 

However, it would **not change** the **worst-case complexity** of the algorithm:

  * in the worst case each element of **L** is examined.

### <font color="blue">Binary search</font>

We can get a considerable **improvement** in the **worst-case complexity** by using an algorithm, 

* <b>binary search</b>,

Here we rely on the **assumption** that the list is **ordered**.

The idea is simple:

* Pick an index, i, that divides the list L <b>roughly in half</b>.

* Ask if L[i] == e.

* If not, ask whether **L[i]** is larger or smaller than **e**.

* Depending upon the answer, search either <b>the left or right half</b> of **L** for **e**.


In [7]:
# #Page 129, Figure 10.2
def search(L, e):
    """Assumes L is a list, the elements of which are in
          ascending order.
       Returns True if e is in L and False otherwise"""
    
    def bSearch(L, e, low, high):
        #Decrements high - low
        if high == low:
            return L[low] == e
        mid = (low + high)//2  #  i roughly in half of list. 
        if L[mid] == e:
            return True
        elif L[mid] > e:
            if low == mid: #nothing left to search
                return False
            else:
                return bSearch(L, e, low, mid - 1)# left
        else:
            return bSearch(L, e, mid + 1, high)  # right
        
    if len(L) == 0:
        return False
    else:
        return bSearch(L, e, 0, len(L) - 1)

The specification says that the implementation may assume that

* **L** is <b>sorted in ascending order</b>

### The complexity of **bSearch.** 

The complexity of **bSearch** depends only upon 

* <b>the number of <font color='blue'>recursive</font> calls</b>.

The question is 

* **how many times** can the value of **high–low** be cut in half before **high–low == 0?**

$2^?=high-low$

then:

$?=\log_2 ^{(high-low)}$

so, <b>high–low</b> can be cut in half at most <b>$\log_2^{(high–low)}$</b> times before it reaches 0.

The complexity of search is <b>O(log(len(L)))</b>.

In [31]:
L1 = [1, 4, 5, 8, 12, 19, 24, 31, 43, 55]
print(search(L1, 8))
print(search(L1, 12))
print(search(L1, 24))
# 
print(search(L1, 21))

True
True
True
False


### Binary Search in C

In [26]:
%%file ./code/ds/bSearch.h

/* Search an array for a key using Binary Search */

#ifndef BSEARCH_H
#define BSEARCH_H


int bSearch(const int a[], int size, int key);

#endif

Overwriting ./code/ds/bSearch.h


In [34]:
%%file ./code/ds/bSearch.c

/* Search an array for a key using Binary Search (BinarySearch.c) */

#include "bSearch.h"
#include <stdio.h>
 
int binarySearch(const int a[], int iLeft, int iRight, int key);
void print(const int a[], int iLeft, int iRight);
  
// Search the array for the given key
// If found, return array index; otherwise, return -1
int bSearch(const int a[], int size, int key) {
   // Call recursive helper function
   return binarySearch(a, 0, size-1, key);
}
 
// Recursive helper function for binarySearch
int binarySearch(const int a[], int iLeft, int iRight, int key) {
  
   // For tracing the algorithm
   print(a, iLeft, iRight);
 
   // Test for empty list
   if (iLeft > iRight) return -1;
 
   // Compare with middle element
   int mid = (iRight + iLeft) / 2;  // truncate
   if (key == a[mid]) {
      return mid;
   } else if (key < a[mid]) {
      // Recursively search the lower half
      binarySearch(a, iLeft, mid - 1, key);
   } else {
      // Recursively search the upper half
      binarySearch(a, mid + 1, iRight, key);
   }
}

// Print the contents of the given array from iLeft to iRight (inclusive)
void print(const int a[], int iLeft, int iRight) {
   printf("{");
   for (int i = iLeft; i <= iRight; ++i) {
      printf("%d",a[i]);
      if (i < iRight) printf(",");
   }
   printf("} \n");
}

Overwriting ./code/ds/bSearch.c


In [4]:
%%file ./code/ds/DemobSearch.c

#include <stdio.h>
#include <stdlib.h> 
#include "bSearch.h" 

int main() {
   const int SIZE = 10;
   int a1[10] = {1, 4, 5, 8, 12, 19, 24, 31, 43, 55}; // sorted
 
   int keys[4]={8,12,24,21};
   for(int i=0; i<4; i++) 
       printf("%d's index is: %d \n",keys[i],bSearch(a1,  SIZE, keys[i]));
   
   return 0; 
}

Overwriting ./code/ds/DemobSearch.c


In [5]:
!gcc -c ./code/ds/DemobSearch.c ./code/ds/bSearch.c
!gcc -o ./code/ds/DemobSearch.exe DemobSearch.o bSearch.o

In [6]:
!.\code\ds\DemobSearch.exe

{1,4,5,8,12,19,24,31,43,55} 
{1,4,5,8} 
{5,8} 
{8} 
8's index is: 3 
{1,4,5,8,12,19,24,31,43,55} 
12's index is: 4 
{1,4,5,8,12,19,24,31,43,55} 
{19,24,31,43,55} 
{19,24} 
{24} 
24's index is: 6 
{1,4,5,8,12,19,24,31,43,55} 
{19,24,31,43,55} 
{19,24} 
{24} 
{} 
21's index is: -1 


In [26]:
%%file makefile

all: DemobSearch.exe

DemobSearch.exe: bSearchobjs
	 gcc -o ./code/ds/DemobSearch.exe DemobSearch.o bSearch.o
	 del DemobSearch.o
    
bSearchobjs: 
	 gcc -c -I./code/ds ./code/ds/DemobSearch.c ./code/ds/bSearch.c
     
clean:
	 del .\code\ds\DemobSearch.exe

Overwriting makefile


In [27]:
!make

gcc -c -I./code/ds ./code/ds/DemobSearch.c ./code/ds/bSearch.c
gcc -o ./code/ds/DemobSearch.exe DemobSearch.o bSearch.o
del DemobSearch.o


In [28]:
!.\code\ds\DemobSearch.exe

8's index is: 3 
12's index is: 4 
24's index is: 6 
21's index is: -1 


### GNU C Linrary : Array Search Function

http://www.gnu.org/software/libc/manual/html_node/Array-Search-Function.html#Array-Search-Function

The GNU C Library contains functions to perform **linear** search. The prototypes for the following two functions can be found in **search.h**. 
```c
void * lfind (const void *key, const void *base, size_t *nmemb, size_t size, comparison_fn_t compar)
void * lsearch (const void *key, void *base, size_t *nmemb, size_t size, comparison_fn_t compar)
```

The **lfind()** function shall be equivalent to lsearch(), except that if the entry is **not found**, it is not added to the table. Instead, a **null pointer** is returned.

The **lsearch()** function shall linearly search the table and return a pointer into the table for the matching entry. If the entry does **not occur**, it shall be **added** at the end of the table,it **increments** the value of <b>*nmemb</b> to reflect this addition. 

To do this, you supply **a comparison function** to compare two elements of the array

This type is a GNU extension
```c
int comparison_fn_t (const void *, const void *);
```

In [164]:
%%file ./code/ds/gnulsearch.c

/* linear searching an array using The GNU C Library ：lsearch (gnulsearch.c) */

#include <stdio.h>
#include <stdlib.h> 
#include <search.h> 

int compare_ints (const void *a, const void *b)
{
   return (*(int*)a - *(int*)b);  
}

int main() {
   unsigned int SIZE = 8;
   int values[8] = {8, 4, 5, 3, 2, 9, 4, 1};
   int keys[3]={8,4,99};
   int *pItem; 
   int i;
   for(i=0; i<3; i++) 
   { 
      pItem = (int*)lfind(&keys[i],&values[0],&SIZE,sizeof(int),compare_ints);
      if (pItem!=NULL)
         printf("%d is in the array \n",*pItem);
      else  
        printf ("%d is not in the array \n",keys[i]);  
   }
   int rawSIZE=SIZE; 
   for(i=0; i<3; i++) 
   { 
      pItem = (int*)lsearch(&keys[i],&values[0],&SIZE,sizeof(int),compare_ints);
      if (SIZE>rawSIZE)
      {
         printf("%d is added into the array \n",*pItem);
         rawSIZE=SIZE;
      }    
      else 
        printf("%d is in the array \n",*pItem);
   }
   return 0;
}
    

Overwriting ./code/ds/gnulsearch.c


In [162]:
!gcc -o ./code/ds/gnulsearch.exe ./code/ds/gnulsearch.c

In [163]:
!.\code\ds\gnulsearch.exe

8 is in the array 
4 is in the array 
99 is not in the array 
8 is inthe array 
4 is inthe array 
99 is added into the array 


### GNU C Library: bsearch

To search a sorted array for an element matching the key, use the **bsearch** function. The prototype for this function is in the header file **stdlib.h** 
```c
void * bsearch (const void *key, const void *array, size_t count, size_t size, comparison_fn_t compare)
```
The return value is a pointer to the matching array element, or a null pointer if no match is found. If the array contains more than one element that matches, the one that is returned is unspecified.

In [132]:
%%file ./code/ds/gnubsearch.c

/* binary searching an array using The GNU C Library ：bsearch (gnubsearch.c) */

#include <stdio.h>  
#include <stdlib.h>  
  
int compare_ints (const void * a, const void * b)  
{  
  return ( *(int*)a - *(int*)b );  
}  
  
int main ()  
{  
  int values[] = { 10, 20, 25, 40, 90, 100 };  
  int * pItem;  
  int key = 400;  
  pItem = (int*) bsearch (&key, values, 6, sizeof (int), compare_ints);  
  if (pItem!=NULL)  
    printf ("%d is in the array.\n",*pItem);  
  else  
    printf ("%d is not in the array.\n",key);  
  return 0;  
}  

Overwriting ./code/ds/gnubsearch.c


In [133]:
!gcc -o ./code/ds/gnubsearch.exe ./code/ds/gnubsearch.c

In [134]:
!.\code\ds\gnubsearch.exe

400 is not in the array.


## 10.2 Sorting Algorithms

the standard implementation of sorting in most Python implementations runs in roughly $O(n*log(n))$ time, where n is the length of the list.

In most cases, the right thing to do is to use either Python’s built-in sort method:

```python
L.sort()
``` 

sorts the list L(ascending sort)


In [34]:
L=[1,34,5,7,90,2]
L.sort()
print('sorted L',L)
L.reverse()
print('reversed L',L)

sorted L [1, 2, 5, 7, 34, 90]
reversed L [90, 34, 7, 5, 2, 1]


In [36]:
L =[1,34,5,7,90,2]
L1 = L[ : ]
# descending sorts
L1.sort(reverse = True)
print(L)
print(L1)

[1, 34, 5, 7, 90, 2]
[90, 34, 7, 5, 2, 1]


its built-in function ```sorted(L)``` returns a **new** list with same elements as L, but <b>does not mutate L</b>

In [38]:
L=[1,34,5,7,90,2]
# returns a new list with same elements as L
L1=sorted(L)
print(L)
print(L1)

[1, 34, 5, 7, 90, 2]
[1, 2, 5, 7, 34, 90]


In [ ]:
L=[1,34,5,7,90,2]
L1=sorted(L,reverse = True)
print(L)
print(L1)

present sorting algorithms here primarily to provide some practice in thinking about algorithm design and complexity analysis

### Selection sort 

**Selection sort** works by maintaining the **loop invariant** that, given a partitioning of the list into 

* <b>a prefix (L[0:i])</b> 

* <b>a suffix (L[i+1:len(L)])</b>,

the prefix is sorted and no element in the prefix is larger than <b>the smallest element</b> in the suffix.

We use induction to reason about **loop invariants**.

* **Base case**: At the **start** of the first iteration, the **prefix** is **empty**, i.e., the
**suffix** is the **entire list**. The invariant is (trivially) true.


* **Induction step**: At each step of the algorithm, we move one element from the suffix to the prefix. We do this by appending a minimum element of the suffix to the end of the prefix. Because the invariant held before we moved the element, we know that after we append the element the prefix is still sorted. We also know that since we removed the smallest element in the suffix, no element in the prefix is larger than the smallest element in the suffix.


* When **the loop is exited**, the **prefix** includes the **entire list**, and the **suffix**
is **empty**.


Therefore, the entire list is now sorted in ascending order.

In [40]:
#Page 132, Figure 10.3
def selSort(L):
    """Assumes that L is a list of elements that can be
         compared using >.
       Sorts L in ascending order"""
    suffixStart = 0
    while suffixStart != len(L):
        #look at each element in suffix
        for i in range(suffixStart, len(L)):
            if L[i] < L[suffixStart]:
                #swap position of elements
                L[suffixStart], L[i] = L[i], L[suffixStart]
        suffixStart += 1

In [41]:
L=[1,34,5,7,90,2]
selSort(L)
print(L)

[1, 2, 5, 7, 34, 90]


Unfortunately, it is rather inefficient

* The complexity of the inner loop is $O(len(L))$

* The complexity of the outer loop is also $O(len(L))$.

The complexity of the entire function is $O(len(L)^2)$. I.e., it is **quadratic** in the length of L.

In [7]:
%%file ./code/ds/SelectionSort.c

/* Sorting an array using Selection Sort (SelectionSort.c) */

#include <stdio.h>
#include <stdlib.h> 

void selectionSort(int a[], int size);
void print(const int a[], int iMin, int iMax);

// Sort the given array of size using selection sort
void selectionSort(int a[], int size) {
   int temp; // for swaping
   for (int i = 0; i < size - 1; ++i) {
      // for tracing
      print(a, 0, i - 1);
      print(a, i, size - 1);
 
      // [0, i-1] already sort
      // Search for the smallest element in [i, size-1]
      //  and swap with a[i]
      int minIndex = i;  // assume fist element is the smallest
      for (int j = i + 1; j < size; ++j) {
         if (a[j] < a[minIndex]) minIndex = j;
      }
      if (minIndex != i) {  // swap
         temp = a[i];
         a[i] = a[minIndex];
         a[minIndex] = temp;
      }
 
      // for tracing
      printf("=> ");
      print(a, 0, i - 1);
      print(a, i, size - 1);
      printf("\n");
   }
}
 
// Print the contents of the array in [iMin, iMax]
void print(const int a[], int iMin, int iMax) {
   printf("{");
   for (int i = iMin; i <= iMax; ++i) {
      printf("%d",a[i]);
      if (i < iMax)  printf(",");
   }
   printf("}");
}
 
int main() {
   const int SIZE = 8;
   int a[8] = {8, 4, 5, 3, 2, 9, 4, 1};
   print(a, 0, SIZE - 1);
   printf("\n"); 
   selectionSort(a, SIZE);
   print(a, 0, SIZE - 1);
   printf("\n"); 
  
   return 0;
}


Overwriting ./code/ds/SelectionSort.c


In [8]:
!gcc -o ./code/ds/SelectionSort.exe ./code/ds/SelectionSort.c

In [9]:
!.\code\ds\SelectionSort.exe

{8,4,5,3,2,9,4,1}
{}{8,4,5,3,2,9,4,1}=> {}{1,4,5,3,2,9,4,8}
{1}{4,5,3,2,9,4,8}=> {1}{2,5,3,4,9,4,8}
{1,2}{5,3,4,9,4,8}=> {1,2}{3,5,4,9,4,8}
{1,2,3}{5,4,9,4,8}=> {1,2,3}{4,5,9,4,8}
{1,2,3,4}{5,9,4,8}=> {1,2,3,4}{4,9,5,8}
{1,2,3,4,4}{9,5,8}=> {1,2,3,4,4}{5,9,8}
{1,2,3,4,4,5}{9,8}=> {1,2,3,4,4,5}{8,9}
{1,2,3,4,4,5,8,9}


### 10.2.1 Merge Sort

Merge sort is a prototypical <b>divide-and-conquer algorithm</b>. It was invented in 1945, by John von Neumann, and is still widely used.

In general, a divide-and-conquer algorithm is characterized by

* 1 A **threshold input size**, below which the problem is **not subdivided**

* 2 The size and number of **sub-instances** into which an instance is **split**,

* 3 The algorithm used to **combine** sub-solutions.


The threshold is sometimes called the **recursive base**. For item 2 it is usual to consider the ratio of initial problem size to sub-instance size. In most of the examples we’ve seen so far, the ratio was 2.

Like many divide-and-conquer algorithms it is most easily described recursively:

* If the list is of **length 0 or 1**, it is **already** sorted.  
 
     A **threshold input size**
     

* If the list has more than one element, **split the list** into two lists, and use **merge sort** to sort each of them.

    The size and number of **sub-instances** into which an instance is **split**,



* **Merge** the results.

   The algorithm used to **combine** sub-solutions.


<font color='blue'>**The idea**</font> is: 

>look at the **first** element of each list, and move the **smaller of the two** to the **end** of the result list.
>
>When one of the lists is empty, all that remains is to copy the remaining items from the other list.

Consider, for example, merging the two lists
```
 [1,5,12,18,19,20]
 [2,3,4,17]
```
<img src="./img/mergesort.PNG"/>

### What is the complexity of the merge process? 

It involves two constant-time operations： 

* 1 comparing the values of elements 

* 2 copying elements from one list to another. 

The number of comparisons is **O(len(L))**, where L is the **longer** of the two lists. 

The number of copy operations is **O(len(L1) + len(L2))**, because each element gets copied exactly once. 

Therefore, merging two sorted lists is linear in **the length of the lists**

In [1]:
#Page 134, Figure 10.4
def merge(left, right, compare):
    """Assumes left and right are sorted lists and
         compare defines an ordering on the elements.
       Returns a new sorted (by compare) list containing the
         same elements as (left + right) would contain."""
    
    result = []
    i,j = 0, 0
    while i < len(left) and j < len(right):
        if compare(left[i], right[j]):
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    while (i < len(left)):
        result.append(left[i])
        i += 1
    while (j < len(right)):
        result.append(right[j])
        j += 1
    return result

import operator
#  compare = operator.lt
def mergeSort(L, compare = operator.lt):
    """Assumes L is a list, compare defines an ordering
         on elements of L
       Returns a new sorted list containing the same elements as L"""
    if len(L) < 2:
        return L[:]
    else:
        middle = len(L)//2
        left = mergeSort(L[:middle], compare)
        right = mergeSort(L[middle:], compare)
        return merge(left, right, compare)

In [2]:
L=[1,5,12,18,19,20,2,3,4,17]
L1=mergeSort(L,operator.gt)
print(L1)

[20, 19, 18, 17, 12, 5, 4, 3, 2, 1]


#### Let’s analyze the complexity of mergeSort. 

We already know that the time complexity of merge is $O(len(L))$. At each level of recursion the total number of elements to be merged is $len(L)$. 

Therefore, the time complexity of mergeSort is

* $O(len(L))$ multiplied by the number of levels of recursion. 

Since mergeSort divides the list <b>in half</b> each time, we know that the number of levels of recursion is $O(log(len(L))$. 
  
Therefore, the time complexity of mergeSort is $O(n*log(n))</b>$, where n is len(L).
                                                                                                             
This improvement in time complexity comes with a price. 

<b>Selection sort</b> is an example of an <b>in-place</b> sorting algorithm. Because it works by swapping the place of elements within the list, it uses only <b>a constant amount of extra storage</b> (one element in our implementation). 

In contrast, the <b>merge sort</b> algorithm  involves making<b> copies of the list</b>. This means that its space complexity is $O(len(L))$.                                                                                                             

**A merge sort** works as follows:

* Divide the unsorted list into n sublists, each containing 1 element (a list of 1 element is considered sorted).

* Repeatedly merge sublists to produce new sorted sublists until there is only 1 sublist remaining. This will be the sorted list.

In [41]:
%%file ./code/ds/MergeSort.c

/* Sorting an array using Merge Sort (MergeSort.c) */
#include <stdio.h>
#include <stdlib.h>
 
void mSort(int a[], int size);
void mergeSort(int a[], int iLeft, int iRight, int work[]);
void merge(int a[], int iLeftHalfLeft, int iLeftHalfRight,
           int iRightHalfLeft, int iRightHalfRight, int work[]);
void print(const int a[], int iLeft, int iRight);

 
// Sort the given array of size
void mSort(int a[], int size) {
   int work[size];  // work space
   mergeSort(a, 0, size - 1, work);
}
 
// Sort the given array in [iLeft, iRight]
void mergeSort(int a[], int iLeft, int iRight, int work[]) {
   if ((iRight - iLeft) >= 1) {   // more than 1 elements, divide and sort
      // Divide into left and right half
      int iLeftHalfLeft = iLeft;
      int iLeftHalfRight = (iRight + iLeft) / 2;   // truncate
      int iRightHalfLeft = iLeftHalfRight + 1;
      int iRightHalfRight = iRight;
 
      // Recursively sort each half
      mergeSort(a, iLeftHalfLeft, iLeftHalfRight, work);
      mergeSort(a, iRightHalfLeft, iRightHalfRight, work);
 
      // Merge two halves
      merge(a, iLeftHalfLeft, iLeftHalfRight, iRightHalfLeft, iRightHalfRight, work);
   }
}
 
// Merge two halves in [iLeftHalfLeft, iLeftHalfRight] and [iRightHalfLeft, iRightHalfRight]
// Assume that iLeftHalfRight + 1 = iRightHalfLeft
void merge(int a[], int iLeftHalfLeft, int iLeftHalfRight,
           int iRightHalfLeft, int iRightHalfRight, int work[]) {
   int size = iRightHalfRight - iLeftHalfLeft + 1;
   int iResult = 0;
   int iLeft = iLeftHalfLeft;
   int iRight = iRightHalfLeft;
   while (iLeft <= iLeftHalfRight && iRight <= iRightHalfRight) {
      if (a[iLeft] <= a[iRight]) {
         work[iResult++] = a[iLeft++];
      } else {
         work[iResult++] = a[iRight++];
      }
   }
   // Copy the remaining left or right into work
   while (iLeft <= iLeftHalfRight) work[iResult++] = a[iLeft++];
   while (iRight <= iRightHalfRight) work[iResult++] = a[iRight++];
 
   // for tracing
   print(a, iLeftHalfLeft, iLeftHalfRight);
   print(a, iRightHalfLeft, iRightHalfRight);
   printf("=> ");
   print(work, 0, size - 1);
   printf("\n");
 
   // Copy the work back to the original array
   for (iResult = 0, iLeft = iLeftHalfLeft; iResult < size; ++iResult, ++iLeft) {
      a[iLeft] = work[iResult];
   }
}
 
// Print the contents of the given array from iLeft to iRight (inclusive)
void print(const int a[], int iLeft, int iRight) {
   printf("{");
   for (int i = iLeft; i <= iRight; ++i) {
      printf("%d", a[i]);
      if (i < iRight) printf(",");
   }
   printf("} ");
}

 
int main() {
   // Test 1
   const int SIZE_1 = 8;
   int a1[8] = {8, 4, 5, 3, 2, 9, 4, 1};
 
   print(a1, 0, SIZE_1 - 1);
   printf("\n");
   mSort(a1, SIZE_1);
   print(a1, 0, SIZE_1 - 1);
   printf("\n \n");
 
   // Test 2
   const int SIZE_2 = 13;
   int a2[13] = {8, 4, 5, 3, 2, 9, 4, 1, 9, 1, 2, 4, 5};
 
   print(a2, 0, SIZE_2 - 1);
   printf("\n");
   mSort(a2, SIZE_2);
   print(a2, 0, SIZE_2 - 1);
   printf("\n \n");
    
   return 0;
}

Overwriting ./code/ds/MergeSort.c


In [42]:
!gcc -o ./code/ds/MergeSort.exe ./code/ds/MergeSort.c

In [43]:
!.\code\ds\MergeSort.exe

{8,4,5,3,2,9,4,1} 
{8} {4} => {4,8} 
{5} {3} => {3,5} 
{4,8} {3,5} => {3,4,5,8} 
{2} {9} => {2,9} 
{4} {1} => {1,4} 
{2,9} {1,4} => {1,2,4,9} 
{3,4,5,8} {1,2,4,9} => {1,2,3,4,4,5,8,9} 
{1,2,3,4,4,5,8,9} 
 
{8,4,5,3,2,9,4,1,9,1,2,4,5} 
{8} {4} => {4,8} 
{5} {3} => {3,5} 
{4,8} {3,5} => {3,4,5,8} 
{2} {9} => {2,9} 
{2,9} {4} => {2,4,9} 
{3,4,5,8} {2,4,9} => {2,3,4,4,5,8,9} 
{1} {9} => {1,9} 
{1,9} {1} => {1,1,9} 
{2} {4} => {2,4} 
{2,4} {5} => {2,4,5} 
{1,1,9} {2,4,5} => {1,1,2,4,5,9} 
{2,3,4,4,5,8,9} {1,1,2,4,5,9} => {1,1,2,2,3,4,4,4,5,5,8,9,9} 
{1,1,2,2,3,4,4,4,5,5,8,9,9} 
 


### The GNU C Library: Array Sort Function

http://www.gnu.org/software/libc/manual/html_node/Array-Sort-Function.html

To sort an array using an arbitrary comparison function, use the **qsort** function. The prototype for this function is in **stdlib.h**. 

Function:

```c
void qsort (void *array, size_t count, size_t size, comparison_fn_t compare)
```
To do this, you supply **a comparison function** to compare two elements of the array

This type is a GNU extension
```c
int comparison_fn_t (const void *, const void *);
```

In [165]:
%%file ./code/ds/gccqsort.c

/* Sorting an array using The GNU C Library ：Sort (gccqsort.c) */

#include <stdio.h>
#include <stdlib.h> 

int compare_ints (const void * a, const void * b)  
{  
  return ( *(int*)a - *(int*)b );  
}  

int main() {
   const int SIZE = 8;
   int a[8] = {8, 4, 5, 3, 2, 9, 4, 1};
   qsort(a, SIZE, sizeof(int),compare_ints);
   
   for(int i = 0; i <SIZE; i++) 
      printf("%d ", a[i]);
   
  return 0;
}
    

Overwriting ./code/ds/gccqsort.c


In [166]:
!gcc -o ./code/ds/gccqsort.exe ./code/ds/gccqsort.c

In [167]:
!.\code\ds\gccqsort.exe

1 2 3 4 4 5 8 9 



### 10.2.2 Exploiting Functions as Parameters

Suppose we want to sort a list of names written as firstName lastName, e.g.,

the list ['Chris Terman', 'Tom Brady', 'Eric Grimson', 'Gisele Bundchen'].

Figure 10.5 defines two ordering functions, and then uses these to sort a list in two different ways.

In [ ]:
#Page 135, Figure 10.5
def lastNameFirstName(name1, name2):
    name1 = name1.split(' ')
    name2 = name2.split(' ')
    if name1[1] != name2[1]:
        return name1[1] < name2[1]
    else: #last names the same, sort by first name
        return name1[0] < name2[0]

def firstNameLastName(name1, name2):
    # If that argument is omitted 
    #  the string is split using arbitrary strings of whitespace characters
    #  space, tab, newline, return, and formfeed).
    name1 = name1.split()
    name2 = name2.split()
    if name1[0] != name2[0]:
        return name1[0] < name2[0]
    else: #first names the same, sort by last name
        return name1[1] < name2[1]


In [ ]:
L = ['Chris Terman', 'Tom Brady', 'Eric Grimson', 'Gisele Bundchen']
newL = mergeSort(L, lastNameFirstName)
print('Sorted by last name =', newL)
newL = mergeSort(L, firstNameLastName)
print('Sorted by first name =', newL)

### 10.2.3 Sorting in Python

The sorting algorithm used in most Python implementations is called 

* <b>Timsort</b> ： https://en.wikipedia.org/wiki/Timsort

The key idea is to take advantage of the fact that in a lot of data sets the data is <b>already partially sorted</b>. 

**Timsort**’s worst-case performance is the same as merge sort’s, but on average it performs considerably better.

The Python method <b>list.sort</b> takes a list as its first argument and modifies that list
    
The Python function 

**sorted** takes an iterable object (e.g., a list or a dictionary) as its first argument and returns a new sorted list.

In [ ]:
L = [3,5,2]
D = {'a':12, 'c':5, 'b':'dog'}
print(sorted(L))
print(L)
L.sort()
print(L)
print(sorted(D))
D.sort()

Both the list.sort method and the sorted function can have two additional parameters.

The <b>key</b> parameter plays the same role as compare in our implementation of merge sort: it is used to <b>supply the comparison function</b> to be
used.

The <b>reverse</b> parameter specifies whether the list is to be sorted in <b>ascending or descending order</b>.

In [ ]:
L = [[1,2,3], (3,2,1,0), 'abc']
print(sorted(L, key = len, reverse = True))

In [ ]:
def getlastNameFirstName(name):
    name = name.split(' ')
    return name[1]

def getfirstNameLastName(name):
    name = name.split()
    return name[0]

In [ ]:
L = ['Chris Terman', 'Tom Brady', 'Eric Grimson', 'Gisele Bundchen']
newL = sorted(L, key=getlastNameFirstName)
print('Sorted by last name =', newL)
newL =sorted(L, key=getfirstNameLastName)
print('Sorted by first name =', newL)

Both the list.sort method and the sorted function provide <b>stable sorts</b>. 

This means that if two elements are equal with respect to the comparison used in the sort, <b>their relative ordering</b> in the original list (or other iterable object) is <b>preserved<b> in the final list.



## 10.3 Hash Tables

If we put merge sort together with binary search, we have a nice way to search lists.

we can still ask, is logarithmic **the best** that we can do for  search when we are willing to do some

<b>preprocessing</b>

###  When we introduced the type <font color="blue">dict</font> in Chapter 5 

* dictionaries use a technique called <b>hashing</b> to do <b>the lookup in time</b> 

  that is <b>nearly independent of the size of the dictionary</b>.

In [14]:
import timeit

query_lst = [-600,-60,-6,-70,-6,0,6,70,6,60,600]

lst = []
dic = {}

# search space :lst,dic
size=10000
for i in range(size):
    lst.append(i)
    dic[i] = i 

ls="""
for v in query_lst:
    if v in lst:
        continue
"""
   
t_querylst=timeit.Timer(stmt=ls,globals=globals())
lst_time=t_querylst.timeit(10)
print('Linear search time : ',lst_time/10)

bls="""
for v in query_lst:
    if search(lst, v):
       continue
"""
t_querylst=timeit.Timer(stmt=bls,globals=globals())
bst_time=t_querylst.timeit(10)
print('Binary search time : ',bst_time/10)

ds="""
for v in query_lst:
    if v in dic:
        continue
"""
t_querydic=timeit.Timer(stmt=ds,globals=globals())
dict_time=t_querydic.timeit(10)
print('dict search time : ',dict_time/10)

Linear search time :  0.0015883946883832323
Binary search time :  0.00013653307782988123
dict search time :  1.2832056199840736e-06


### A hash table 

The basic idea behind **a hash table** is

* **convert the key to an integer, and then use that integer to index into a list**

  which can be done in constant time.

  In principle, values of any immutable type can be easily converted to an integer through **Hash functions**.


### Hash functions

**Hash functions** can be used to convert 

* <b>a large space of keys</b>  to  <b>a smaller space of integer indices</b>.

Since the space of possible outputs is smaller than the space of possible inputs, 

a hash function is a 

<b>many-to-one mapping</b>,

multiple different inputs may be mapped to the same output. 

When two inputs are mapped to the same output, 

it is called a <b>collision</b>. 

A good hash function produces 

<b>a uniform distribution</b>

every output in the range is equally probable, which <b>minimizes the probability of collisions</b>.

Designing good hash functions is surprisingly challenging.

The problem is that one wants the outputs to be uniformly distributed given the expected distribution of inputs

## hash bucket

**class intDict(object)**  uses a simple hash function 

```python
# i%j returns the remainder when the integer i is divided by the integer j 

hashBucket = self.buckets[dictKey%self.numBuckets]
``` 

to implement a dictionary with integers as keys.

The basic idea is to represent 

**an instance of class intDict**

by a list of <b>hash buckets</b>, 

where **each bucket** is a list of **key/value** pairs. 

<b>By making each bucket a list, 

we handle collisions by storing all of the values that hash to the same bucket in the list</b>. 

The <b>hash table</b> works as follows: 

*  **1 def __init__(self, numBuckets):**

   The instance variable <b>buckets</b> is initialized to 

   <b>a list</b> of  <b>numBuckets</b> <b>empty lists</b>.

```python
        self.buckets = []
        self.numBuckets = numBuckets
        for i in range(numBuckets):
            self.buckets.append([]) 
```
   
* **2 def addEntry(self, dictKey, dictVal):**

   To store or look up an entry with key **dictKey**, 

```python   
        hashBucket = self.buckets[dictKey%self.numBuckets]
        for i in range(len(hashBucket)):
            if hashBucket[i][0] == dictKey:
                hashBucket[i] = (dictKey, dictVal) #if one was found,replace
                return
        hashBucket.append((dictKey, dictVal)) # append a new entry to the bucket if none was found.
```      
   
we use <b>the hash function i%j to convert dictKey into an integer</b>, 
```python  
    hashBucket = self.buckets[dictKey%self.numBuckets
```    
and use that integer to index into buckets 
```python
   hashBucket[i]
```
to find the hash bucket associated with **dictKey**

If <b>a value is to be stored</b>,then  

if one was found:  <b>replace</b> the value in the existing entry, if one was found, 

if none was found: <b>append</b> a new entry to the bucket 

* **3 def getValue(self, dictKey):**

We then search that bucket (which is a list) linearly to see if there is an entry with the key dictKey.

```python 
 for e in hashBucket:
            if e[0] == dictKey: // key
                return e[1]     // valu
```

If we are doing <b>a lookup</b> and there is an entry with the key, we simply return the value stored with that key. 

If there is no entry with that key, we return None. 




In [ ]:
#Page 139, Figure 10.6

class intDict(object):
    """A dictionary with integer keys"""
    
    def __init__(self, numBuckets):
        """Create an empty dictionary"""
        
        ## buckets is initialized to a list of numBuckets empty lists.
        
        self.buckets = []
        self.numBuckets = numBuckets
        for i in range(numBuckets):
            self.buckets.append([]) # empty list
            
    def addEntry(self, dictKey, dictVal):
        """Assumes dictKey an int.  Adds an entry."""
        hashBucket = self.buckets[dictKey%self.numBuckets]
        for i in range(len(hashBucket)):
            if hashBucket[i][0] == dictKey:
                hashBucket[i] = (dictKey, dictVal) #if one was found,replace
                return
        hashBucket.append((dictKey, dictVal)) # append a new entry to the bucket if none was found.
        
    def getValue(self, dictKey):
        """Assumes dictKey an int.  Returns entry associated
           with the key dictKey"""
        hashBucket = self.buckets[dictKey%self.numBuckets]
        for e in hashBucket:
            if e[0] == dictKey: # key
                return e[1]     # value 
        return None
    
    def __str__(self):
        result = '{'
        for b in self.buckets:
            for e in b:
                result = result + str(e[0]) + ':' + str(e[1]) + ','
        return result[:-1] + '}' #result[:-1] omits the last comma



The following code first constructs an **intDict** with twenty entries. 

The values of the entries are the integers 0 to 19.

The **keys** are chosen at random from <b>integers in the range 0 to 10^5 - 1<b>.

## Twenty entries

In [ ]:
import random #a standard library module
Entrys=[]
NumEntry=20

random.seed(1)
for i in range(NumEntry):
    #choose a random int between 0 and 10**5
    key = random.randint(0, 10**5)
    Entrys.append((key,i))

print('The Entrys (key,i)is:')
for entry in Entrys:
    print(entry)

### Search key->value in list of Entrys

In [ ]:
def searchEntrys():
    for entry in Entrys:
        key=entry[0]
        for item in Entrys:
            if (key==item[0]):
                value=item[1] 
                print(key,value)  
                break
searchEntrys()

In [ ]:
def searchEntrys():
    for entry in Entrys:
        if entry in Entrys:
            value=entry[1] 
            key=entry[0]
            print(key,value)  
searchEntrys()

## Put entrys into intDict

* numBuckets =29

In [ ]:
numBuckets =29

D = intDict(numBuckets) # numBuckets >entries <<< the range of key 
for item in Entrys:
    D.addEntry(item[0],item[1])

print('The intDict is:' )
print(D)

print('\n', 'The hase buckets are:')
i=0
for hashBucket in D.buckets:
    print('BucketID',i,'  ', hashBucket)
    i=i+1

we see that many of the hash buckets are **empty**. 

Others contain **one, two, or three tuples** depending upon <b>the number of collisions</b> that occurred

In [ ]:
def getIntDict():
    for entry in Entrys:
        key=entry[0]
        value=D.getValue(key)
        #print(key,value)
getIntDict()        

In [ ]:
%timeit getIntDict()

### What is the complexity of **getValue**? 

If there were <b>no collisions</b> it would be <b>O(1)</b>,  because each hash bucket would be of length 0 or 1.

there might be <b>collisions</b>. 

If everything hashed to the same bucket,

it would be <b>O(n)</b> where n is the number of entries in the dictionary,

because the code would perform a linear search on that hash bucket.

### By making the <b>hash table large enough</b>, 

we can <b>reduce the number of collisions</b> sufficiently to allow us to treat <b>the complexity as O(1)</b>.

### hash table small , more collisions

* numBucket=5

In [ ]:
import random #a standard library module

# hash table small , more collisions
numBuckets=5
D = intDict(numBuckets) # numBuckets < entries

for item in Entrys:
    D.addEntry(item[0],item[1])

print('The value of the intDict is:')
print(D)

print('\n', 'The buckets are:')
for hashBucket in D.buckets: #violates abstraction barrier
    print('  ', hashBucket)


In [ ]:
%timeit getIntDict()

### hash table large , less collisions

* numBucket=50

In [ ]:
import random #a standard library module

# hash table large , less collisions
numBucket=50

D = intDict(numBucket) # numBuckets >> entries

for item in Entrys:
    D.addEntry(item[0],item[1])
    
print('The value of the intDict is:')
print(D)

print('\n', 'The buckets are:')
for hashBucket in D.buckets: #violates abstraction barrier
    print('  ', hashBucket)

In [ ]:
%timeit getIntDict()

##  Note: 

### hash table: a series of buckets

* **bucket** : a list of key/value pairs. storing all of the values that hash to the same bucket in the list 

* **hash each element to bucket**: an address that is derived form the key value by applying the hash function
    
*  **search:**

    * key -> the hash function -> an address of bucket in the hash table -> get valve in bucket

## Further Reading

* 严蔚敏 李冬梅 吴伟民. 数据结构（C语言版），人民邮电出版社（第2版）,2015年2月  


* Mark Allen Weiss. Data Structures and Algorithm Analysis in C


* GNU C Library: Searching and Sorting : http://www.gnu.org/software/libc/manual/html_node/Searching-and-Sorting.html